In [1]:
import sys
sys.path.append('../scripts/')
from kf import *

In [2]:
def make_ax():
    fig = plt.figure(figsize=(4, 4))
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    ax.set_xlabel("X", fontsize=10)
    ax.set_ylabel("Y", fontsize=10)
    return ax

def draw_trajectory(xs, ax):
    poses = [xs[s] for s in range(len(xs))]
    ax.scatter([e[0] for e in poses], [e[1] for e in poses], s=5, marker=".", color="black")
    ax.plot([e[0] for e in poses], [e[1] for e in poses], linewidth=0.5, color="black")
    
def draw_observation(xs, zlist, ax):
    for s in range(len(xs)):
        if s not in zlist:
            continue
            
        for obs in zlist[s]:
            x, y, theta = xs[s]
            ell, phi = obs[1][0], obs[1][1]
            mx = x + ell*math.cos(theta + phi)
            my = y + ell*math.sin(theta + phi)
            ax.plot([x, mx], [y, my], color="pink", alpha=0.5)

def draw_edges(edges, ax):
    for e in edges:
        ax.plot([e.x1[0], e.x2[0]], [e.x1[1], e.x2[1]], color="red", alpha=0.5)
            
def draw(xs, zlist, edges):
    ax = make_ax()
    draw_observation(xs, zlist, ax)
    draw_edges(edges, ax)
    draw_trajectory(xs, ax)
    plt.show()

In [3]:
def read_data():
    hat_xs = {}
    zlist = {}
    
    with open("log.txt") as f:
        for line in f.readlines():
            tmp = line.rstrip().split()
            
            step = int(tmp[1])
            if tmp[0] == "x":
                hat_xs[step] = np.array([float(tmp[2]), float(tmp[3]), float(tmp[4])]).T
            elif tmp[0] == "z":
                if step not in zlist:
                    zlist[step] = []
                zlist[step].append((int(tmp[2]), np.array([float(tmp[3]), float(tmp[4]), float(tmp[5])]).T))
                
        return hat_xs, zlist

In [4]:
class ObsEdge:
    def __init__(self, t1, t2, z1, z2, xs):
        assert z1[0] == z2[0]
        
        self.t1, self.t2 = t1, t2
        self.x1, self.x2 = xs[t1], xs[t2]
        self.z1, self.z2 = z1[1], z2[1]
        
        s1 = math.sin(self.x1[2] + self.z1[1])
        c1 = math.cos(self.x1[2] + self.z1[1])
        s2 = math.sin(self.x2[2] + self.z2[1])
        c2 = math.cos(self.x2[2] + self.z2[1])
        
        hat_e = self.x2 - self.x1 + np.array([
            self.z2[0]*c2 - self.z1[0]*c1,
            self.z2[0]*s2 - self.z1[0]*s1,
            self.z2[1] - self.z2[2] - self.z1[1] + self.z1[2]
        ])
        while hat_e[2] >= math.pi: hat_e[2] -= math.pi*2
        while hat_e[2] < - math.pi: hat_e[2] += math.pi*2
            
        print(hat_e)

In [5]:
import itertools
def make_edges(hat_xs, zlist):
    landmark_keys_zlist = {}
    
    for step in zlist:
        for z in zlist[step]:
            landmark_id = z[0]
            if landmark_id not in landmark_keys_zlist:
                landmark_keys_zlist[landmark_id] = []
                
            landmark_keys_zlist[landmark_id].append((step, z))
            
    edges = []
    for landmark_id in landmark_keys_zlist:
        step_pairs = list(itertools.combinations(landmark_keys_zlist[landmark_id], 2))
        edges += [ObsEdge(xz1[0], xz2[0], xz1[1], xz2[1], hat_xs) for xz1, xz2 in step_pairs]
        
    return edges

In [6]:
hat_xs, zlist = read_data()
edges = make_edges(hat_xs, zlist)
draw(hat_xs, zlist, edges)

[ 0.01150753 -0.11133166 -0.02772057]
[-0.17395507 -0.20323246 -0.10434241]
[-2.04728899 -0.1720091  -0.58285221]
[-2.40459301 -1.49083051 -0.80312898]
[-2.21931814 -0.52309354 -0.66572607]
[-2.32083729 -0.29918278 -0.56128142]
[-2.2339201  -0.44216678 -0.580081  ]
[-2.32935843 -0.42044242 -0.52731497]
[-2.98539825  0.57171814 -0.97319437]
[-2.50459543 -0.03685163 -0.85530065]
[-3.17326748  0.68346034 -1.00399331]
[-2.87463285 -0.16383856 -0.95756353]
[-3.16070249  0.22970934 -1.01558017]
[-2.9855991   0.20954299 -0.99875238]
[-3.22261407  0.29453543 -0.98243387]
[-3.18235159  0.3322827  -0.86255098]
[-3.19007719  0.9841455  -1.16792262]
[-0.1854626  -0.0919008  -0.07662184]
[-2.05879652 -0.06067744 -0.55513164]
[-2.41610054 -1.37949885 -0.77540841]
[-2.23082568 -0.41176188 -0.63800549]
[-2.33234483 -0.18785113 -0.53356084]
[-2.24542764 -0.33083512 -0.55236043]
[-2.34086596 -0.30911076 -0.4995944 ]
[-2.99690578  0.6830498  -0.9454738 ]
[-2.51610296  0.07448003 -0.82758008]
[-3.18477501

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …